# Markdown Cleaner

處理流程：
- 讀取來源 Markdown（保留原標題層級 `#`）
- 移除 `**粗體**` 標記與 `<!-- comment -->` 區塊
- 將其餘標題統一為 `##` 並輸出 `_無粗體` 版本

在 Docker/Jupyter 環境中直接執行下方程式格即可。

In [59]:
from pathlib import Path
import re

BASE_DIR = Path.cwd()
possible_roots = [BASE_DIR, BASE_DIR / 'work'] + list(BASE_DIR.parents) + list((BASE_DIR / 'work').parents)

def resolve_path(relative):
    p = Path(relative)
    if p.is_absolute():
        return p if p.exists() else None
    for base in possible_roots:
        candidate = (base / p).resolve()
        if candidate.exists():
            return candidate
    return None

source_relative = Path('mkdocs/My_Notes/113憲判字11號判決/法官學院書面稿.md')

source_path = resolve_path(source_relative)
if source_path is None:
    raise FileNotFoundError(f'找不到來源檔案：{source_relative}')

output_path = source_path.with_name(source_path.stem + '_無粗體.md')

text = source_path.read_text(encoding='utf-8')
print(f'來源檔案：{source_path}')
print(f'原始字元數：{len(text):,}')

bold_pattern = re.compile(r'\*\*(.+?)\*\*', flags=re.DOTALL)
comment_pattern = re.compile(r'<!--.*?-->', flags=re.DOTALL)
heading_pattern = re.compile(r'^(#+)\s*(.*)$', flags=re.MULTILINE)

processed = bold_pattern.sub(lambda m: m.group(1), text)
processed = comment_pattern.sub('', processed)

def normalize_headings(content: str) -> str:
    def repl(match: re.Match) -> str:
        hashes, title = match.group(1), match.group(2).strip()
        if len(hashes) <= 1:
            return '# ' + title
        return '## ' + title
    return heading_pattern.sub(repl, content)

processed = normalize_headings(processed)

output_path.write_text(processed, encoding='utf-8')
print(f'已輸出：{output_path}')
print(f'新檔字元數：{len(processed):,}')


來源檔案：/home/jovyan/work/mkdocs/My_Notes/113憲判字11號判決/法官學院書面稿.md
原始字元數：29,648
已輸出：/home/jovyan/work/mkdocs/My_Notes/113憲判字11號判決/法官學院書面稿_無粗體.md
新檔字元數：28,968
